In [1]:
from Bio.PDB import PDBParser
import numpy as np

def pdb_to_point_cloud(pdb_file):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("structure", pdb_file)
    point_cloud = []

    for atom in structure.get_atoms():
        coord = atom.coord  # Get x, y, z coordinates
        point_cloud.append(coord)

    return point_cloud


from Bio.PDB.MMCIFParser import MMCIFParser

# Load the CIF file
parser = MMCIFParser()

def cif_to_point_cloud(cif_file):
    structure = parser.get_structure("structure", cif_file)

# Extract atomic coordinates
    coordinates = []
    for model in structure:
        for chain in model:
            for residue in chain:
                for atom in residue:
                    coordinates.append(atom.coord)  # Get Cartesian coordinates
                    
    return coordinates



# Example usage for PDB file
# pdb_file = "../data/4UG0/4ug0_edited.pdb"
# point_cloud = pdb_to_point_cloud(pdb_file)
# point_cloud = np.array(point_cloud)
# print(point_cloud.shape)

# Example usage for CIF
# cif_file = "../data/6WD4/6WD4.cif"

cif_file = "../data/8FC4/8fc4.cif"
#cif_file = "../data/4UG0/4UG0.cif"
point_cloud = cif_to_point_cloud(cif_file)
point_cloud = np.array(point_cloud)
print(point_cloud.shape)

/Users/wenjunzhao/opt/anaconda3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain 1A is discontinuous at line 286365.
  warnings.warn(
/Users/wenjunzhao/opt/anaconda3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain 1B is discontinuous at line 287485.
  warnings.warn(
/Users/wenjunzhao/opt/anaconda3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain 1D is discontinuous at line 287526.
  warnings.warn(
/Users/wenjunzhao/opt/anaconda3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain 1E is discontinuous at line 287542.
  warnings.warn(
/Users/wenjunzhao/opt/anaconda3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain 1F is discontinuous at line 287551.
  warnings.warn(
/Users/wenjunzhao/opt/anaconda3/lib/python3.8/site-packages/Bio/PDB/StructureBui

(297328, 3)


In [9]:
def sample_within_alpha_shape(alpha_shape_mesh, num_samples=4000):
    """
    Samples points within the boundaries of a 3D alpha shape.

    Parameters:
    - alpha_shape_mesh: trimesh.Trimesh
        The alpha shape mesh that defines the 3D boundaries.
    - num_samples: int
        The number of points to sample within the alpha shape.

    Returns:
    - sampled_points: np.ndarray
        Array of sampled points within the alpha shape of shape (num_samples, 3).
    """
    components = alpha_shape_mesh.split(only_watertight=False)  # Get all components
    # Print number of components (only one of them is the shape and all the rest are small pieces due to artifact)
    print( len(components) )
    # Find the largest component based on volume
    alpha_shape_mesh = max(components, key=lambda c: abs(c.volume) )

    # Get the bounding box of the alpha shape
    bbox_min, bbox_max = alpha_shape_mesh.bounds

    # Generate random points within the bounding box
    sampled_points = []
    while len(sampled_points) < num_samples:
        # Generate random points within the bounding box
        points = np.random.uniform(low=bbox_min, high=bbox_max, size=( np.min([num_samples * 2,5000]), 3))

        # Use trimesh's built-in point-in-mesh test to filter points inside the alpha shape
        #print( alpha_shape_mesh.contains(points).shape )
        inside = alpha_shape_mesh.contains(points) #* (not tunnel_mesh.contains(points)[0])

        # Add points that are inside the shape to the sampled list
        sampled_points.extend(points[inside])
        print(len(sampled_points))
    # Return the requested number of samples
    return np.concatenate( (alpha_shape_mesh.vertices,np.array(sampled_points) ) )





In [14]:
import alphashape
import trimesh
import random
random.seed(10)
def alpha_shape_resamp( point_cloud, alpha=0.04 ):
    alpha_shape = alphashape.alphashape(point_cloud,  alpha=alpha)
    new_points = sample_within_alpha_shape(alpha_shape,num_samples=1000)
    alpha_shape_renew = alphashape.alphashape( new_points, alpha)
    print(alpha_shape_renew.is_watertight)
    print('alpha='+str(alpha))
    return alpha_shape_renew

In [15]:
alpha0 = 0.04
alpha1 = 0.01 # This is a fairly big alpha that roughly gives a convex hull. Can even go down to like 0.001.

alpha_shape1 = alpha_shape_resamp( point_cloud, alpha=alpha1 )
alpha_shape0 = alpha_shape_resamp( point_cloud, alpha=alpha0 )

while alpha_shape1.is_watertight == True and alpha_shape0.is_watertight == False and abs(alpha0 - alpha1) > 0.005:
    alpha_new = .5*alpha0 + .5*alpha1
    print(alpha_new)
    # This bisection makes sure that alpha1 gives a watertight mesh...and alpha0 does not.
    alpha_shape_renew = alpha_shape_resamp( point_cloud, alpha=alpha_new )
    if alpha_shape_renew.is_watertight == True:
        alpha1 = alpha_new
        alpha_shape1 = alpha_shape_renew
    else:
        alpha0 = alpha_new
        alpha_shape0 = alpha_shape_renew





76
604
1222
True
alpha=0.01
175
418
881
1323
False
alpha=0.04
0.025
112
499
977
1483
True
alpha=0.025
0.0325
151
455
900
1366
True
alpha=0.0325
0.036250000000000004
156
431
882
1312
True
alpha=0.036250000000000004


In [17]:
alpha_shape_renew.show()

In [19]:
alpha_shape_renew.is_watertight

True

In [20]:
import trimesh

def save_alpha_shape_as_ply(alpha_shape, file_path):
    """
    Save a Trimesh alpha shape as a PLY file.

    Args:
        alpha_shape: A Trimesh object representing the alpha shape.
        file_path: The output file path for the PLY file.
    """
    if not isinstance(alpha_shape, trimesh.Trimesh):
        raise TypeError("avislpha_shape must be a trimesh.Trimesh object.")
    
    # Save as PLYencoding='ascii'
    alpha_shape.export(file_path, file_type='ply',encoding='ascii')
    print(f"Alpha shape saved as PLY file at {file_path}")
save_alpha_shape_as_ply(alpha_shape_renew, "../data/8FC4/alpha_shape_watertight_8FC4.ply")

Alpha shape saved as PLY file at ../data/8FC4/alpha_shape_watertight_8FC4.ply
